# HYPERPARAMETER TUNING WITH WandB
## Uses WandB Sweeps

In [1]:
# Import Depedencies
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import datetime
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Input, Normalization, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TrueNegatives, TruePositives, Precision, Recall, F1Score, AUC
from tensorflow.keras.regularizers import L2
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback, WandbCallback

In [2]:
# Load Malaria Dataset From Tensorflow Datasets
dataset, datasetInfo = tfds.load("malaria", 
                                 with_info=True, 
                                 shuffle_files=True) # -----------++++++    ->    ---+++-++--+++-

# Dataset Split Function
def split(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
    trainDataset = dataset.take(int(TRAIN_RATIO*len(dataset)))
    leftDataset = dataset.skip(int(TRAIN_RATIO*len(dataset)))
    valDataset = leftDataset.take(int(VAL_RATIO*len(dataset)))
    testDataset = leftDataset.skip(int(VAL_RATIO*len(dataset)))
    return trainDataset, valDataset, testDataset

# Split Dataset As 80/10/10 
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1
trainDataset, valDataset, testDataset = split(dataset['train'], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

IMAGE_SIZE = 224
def resizeRescale(inputs):
    return tf.image.resize(inputs['image'], (IMAGE_SIZE, IMAGE_SIZE))/255., inputs['label']
trainDataset = trainDataset.map(resizeRescale)
valDataset = valDataset.map(resizeRescale)
testDataset = testDataset.map(resizeRescale)

# Dataset Batching
BATCH_SIZE=32
BUFFER_SIZE = 8
trainDataset = trainDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valDataset = valDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
testDataset = testDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

2025-01-27 13:45:13.171791: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-01-27 13:45:13.171821: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-01-27 13:45:13.171824: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-01-27 13:45:13.171836: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-27 13:45:13.171846: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Initialize Sweep Configurations

In [3]:
sweepConfig = {
    "name" : "Model Tune",
    "method" : "random", # random, grid or bayes
    "metric" : {'name': 'val_binary_accuracy', 'goal': 'maximize'},
    "parameters" : {
        "N_DENSE_1" : {"values" : [16, 32, 64, 128]},
        "N_DENSE_2" : {"values" : [16, 32, 64, 128]},
        "LEARNING_RATE" : {"distribution":"uniform", "min" : 0.0001, "max" : 0.1},
        "DROPOUT_RATE" : {"distribution":"uniform", "min" : 0.1, "max" : 0.4},
        "REGULARIZATION_RATE" : {"distribution":"uniform", "min" : 0.001, "max" : 0.1},
    },
}

sweepID = wandb.sweep(sweepConfig, project="Malaria-Detection-HP-Tune")

Create sweep with ID: sisjf06a
Sweep URL: https://wandb.ai/amanjn2003-santa-clara-university/Malaria-Detection-HP-Tune/sweeps/sisjf06a


## Model Definition

In [4]:
CONFIGURATION = {
    "LEARNING_RATE" : 0.001,
    "N_EPOCHS" : 10,
    "BATCH_SIZE" : 128,
    "DROPOUT_RATE": 0.0,
    "IM_SIZE" : 224,
    "REGULARIZATION_RATE" : 0.0,
    "N_FILTERS" : 6,
    "KERNEL_SIZE" : 3,
    "N_STRIDES" : 1,
    "POOL_SIZE" : 2,
    "N_DENSE_1" : 128,
    "N_DENSE_2" : 32
}

def makeModel(config):
    # Using Sequential API with Hyperparamters
    IMAGE_SIZE = 224
    model = tf.keras.Sequential([
        Input(shape=(config.IM_SIZE, config.IM_SIZE, 3)),

        Conv2D(filters=config.N_FILTERS, kernel_size=config.KERNEL_SIZE, strides=config.N_STRIDES, padding='valid', activation='relu', kernel_regularizer=L2(config.REGULARIZATION_RATE)),
        BatchNormalization(),
        MaxPooling2D(pool_size=2, strides=2),
        Dropout(rate = config.DROPOUT_RATE),

        Conv2D(filters=config.N_FILTERS, kernel_size=config.KERNEL_SIZE, strides=config.N_STRIDES, padding='valid', activation='relu', kernel_regularizer=L2(config.REGULARIZATION_RATE)),
        BatchNormalization(),
        MaxPooling2D(pool_size=2, strides=2),

        Flatten(),

        Dense(config.N_DENSE_1, activation='relu', kernel_regularizer=L2(config.REGULARIZATION_RATE)),
        BatchNormalization(),
        Dropout(rate = config.DROPOUT_RATE),

        Dense(config.N_DENSE_2, activation='relu', kernel_regularizer=L2(config.REGULARIZATION_RATE)),
        BatchNormalization(),
        Dropout(rate = config.DROPOUT_RATE),

        Dense(1, activation='sigmoid'),
    ])

    return model

Metrics = [BinaryAccuracy(), FalsePositives(), FalseNegatives(), TrueNegatives(), TruePositives(), Precision(), Recall(), AUC()]

## Run An Agent

### 1. Define a function to run taining based on hyperparamters in sweep configuration

In [5]:
def train(config=None):
    with wandb.init(name="Run-HP-Tune", project="Malaria-Detection", entity="amanjn2003-santa-clara-university", config=CONFIGURATION) as run:
        config = wandb.config
        model = makeModel(config)
        model.compile(
            optimizer = tf.keras.optimizers.Adam(learning_rate = config.LEARNING_RATE),
            loss = tf.keras.losses.BinaryCrossentropy(),
            metrics = Metrics,
        )
        history = model.fit(trainDataset, validation_data=valDataset, epochs=config.N_EPOCHS, callbacks=[WandbMetricsLogger(), WandbModelCheckpoint("Models/TunedModel.keras")])
        wandb.log({"val_binary_accuracy": history.history["val_binary_accuracy"][-1]})

### 2. Pass that function, plus the sweepID to wandb.agent

In [ ]:
count = 20 # Number of Runs
wandb.agent(sweepID, function=train, count=count)

wandb: Agent Starting Run: w2jiu27v with config:
wandb: 	DROPOUT_RATE: 0.1971202749052973
wandb: 	LEARNING_RATE: 0.03884835560459322
wandb: 	N_DENSE_1: 32
wandb: 	N_DENSE_2: 16
wandb: 	REGULARIZATION_RATE: 0.07302866225235208
wandb: Currently logged in as: amanjn2003 (amanjn2003-santa-clara-university). Use `wandb login --relogin` to force relogin


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 1/10


2025-01-27 13:45:16.484538: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


689/689 ━━━━━━━━━━━━━━━━━━━━ 36s 49ms/step - auc: 0.5376 - binary_accuracy: 0.5249 - false_negatives: 2643.4856 - false_positives: 2656.6204 - loss: 4.0146 - precision: 0.5270 - recall: 0.5399 - true_negatives: 2862.1072 - true_positives: 2893.7349 - val_auc: 0.5000 - val_binary_accuracy: 0.4947 - val_false_negatives: 1392.0000 - val_false_positives: 0.0000e+00 - val_loss: 0.6943 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_true_negatives: 1363.0000 - val_true_positives: 0.0000e+00
Epoch 2/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 32s 46ms/step - auc: 0.4904 - binary_accuracy: 0.4989 - false_negatives: 2491.9912 - false_positives: 3075.4841 - loss: 0.6947 - precision: 0.5010 - recall: 0.5935 - true_negatives: 2443.2087 - true_positives: 3045.2637 - val_auc: 0.5000 - val_binary_accuracy: 0.5053 - val_false_negatives: 0.0000e+00 - val_false_positives: 1363.0000 - val_loss: 0.6947 - val_precision: 0.5053 - val_recall: 1.0000 - val_true_negatives: 0.0000e+00 - val_true_positives: 1392.

...

wandb: Agent Starting Run: 4ogypsuy with config:
wandb: 	DROPOUT_RATE: 0.12429993697282306
wandb: 	LEARNING_RATE: 0.03199381526273406
wandb: 	N_DENSE_1: 64
wandb: 	N_DENSE_2: 128
wandb: 	REGULARIZATION_RATE: 0.0116823239225926


Epoch 1/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 38s 51ms/step - auc: 0.5510 - binary_accuracy: 0.5465 - false_negatives: 3782.4275 - false_positives: 2415.3232 - loss: 3.5084 - precision: 0.5672 - recall: 0.4131 - true_negatives: 4471.4131 - true_positives: 3141.7842 - val_auc: 0.6236 - val_binary_accuracy: 0.5053 - val_false_negatives: 0.0000e+00 - val_false_positives: 1363.0000 - val_loss: 0.7997 - val_precision: 0.5053 - val_recall: 1.0000 - val_true_negatives: 0.0000e+00 - val_true_positives: 1392.0000
Epoch 2/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 34s 50ms/step - auc: 0.5899 - binary_accuracy: 0.5639 - false_negatives: 2419.3101 - false_positives: 2494.0913 - loss: 0.9711 - precision: 0.5654 - recall: 0.5766 - true_negatives: 3024.7290 - true_positives: 3117.8174 - val_auc: 0.5000 - val_binary_accuracy: 0.5053 - val_false_negatives: 0.0000e+00 - val_false_positives: 1363.0000 - val_loss: 0.8072 - val_precision: 0.5053 - val_recall: 1.0000 - val_true_negatives: 0.0000e+00 - val_true_positives: 13

epoch/auc,██▅▂▁▁▁▂▁▁
epoch/binary_accuracy,█▇▅▂▁▂▂▂▁▁
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/false_negatives,█▁▃▆▅▆▅▄▇▅
epoch/false_positives,▁▃▄▄█▄▇█▅█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▄▁▁▁▁▁▁▁
epoch/precision,█▆▄▂▁▂▂▂▁▁
epoch/recall,▄█▅▁▄▁▃▄▁▃
epoch/true_negatives,█▃▂▂▁▂▁▁▂▁
epoch/true_positives,█▇▄▁▃▁▃▄▁▃


wandb: Agent Starting Run: dgcsfj3x with config:
wandb: 	DROPOUT_RATE: 0.3908311976704627
wandb: 	LEARNING_RATE: 0.02403442083969619
wandb: 	N_DENSE_1: 64
wandb: 	N_DENSE_2: 64
wandb: 	REGULARIZATION_RATE: 0.023399418716628866


Epoch 1/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 39s 52ms/step - auc: 0.5632 - binary_accuracy: 0.5425 - false_negatives: 2474.7014 - false_positives: 3814.9363 - loss: 3.7622 - precision: 0.5370 - recall: 0.6786 - true_negatives: 3062.8218 - true_positives: 4458.4883 - val_auc: 0.5559 - val_binary_accuracy: 0.5064 - val_false_negatives: 27.0000 - val_false_positives: 1333.0000 - val_loss: 0.8900 - val_precision: 0.5067 - val_recall: 0.9807 - val_true_negatives: 26.0000 - val_true_positives: 1369.0000
Epoch 2/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 40s 57ms/step - auc: 0.5390 - binary_accuracy: 0.5296 - false_negatives: 2620.8752 - false_positives: 2616.6506 - loss: 0.9289 - precision: 0.5306 - recall: 0.5581 - true_negatives: 2902.1174 - true_positives: 2916.3044 - val_auc: 0.5069 - val_binary_accuracy: 0.5053 - val_false_negatives: 0.0000e+00 - val_false_positives: 1363.0000 - val_loss: 0.7964 - val_precision: 0.5053 - val_recall: 1.0000 - val_true_negatives: 0.0000e+00 - val_true_positives: 1392.000

In [7]:
wandb.finish()